In [1]:
from greensight.utils import DIR_DATA
import pandas as pd
import numpy as np
import json
import pandas as pd

In [2]:
# load DataFrame
s2_path = DIR_DATA / "processed_data/sentinel_two_df.hdf"

df_data = pd.read_hdf(s2_path, key="df")
df_orig = df_data.copy()
df_data

greenbelt  Bath and Bristol                                                    \
shape             E06000022                                                     
band                     B1          B11          B12          B2          B3   
2017-04-01       234.805820  1894.865456  1020.251965  311.276690  623.238776   
2017-05-01       216.930759  1909.530406   991.962320  302.160542  635.020438   
2017-06-01       252.662771  2188.810410  1161.614115  340.702544  724.435485   
2017-07-01       300.029123  2017.795291  1048.994736  363.510906  677.887598   
2017-08-01              NaN          NaN          NaN         NaN         NaN   
...                     ...          ...          ...         ...         ...   
2024-06-01       395.767116  2035.439397  1110.330183  454.728518  773.270753   
2024-07-01       293.769673  2288.305522  1249.925666  421.786959  735.594935   
2024-08-01       478.499272  2250.335482  1255.271394  531.902502  793.528614   
2024-09-01       387.997932  2132.064292  1260.179600  507.951894  759.827593   
2024-10-01       342.644046  1924.061259  1071.607682  388.586222  660.778449   

greenbelt                                                                   \
shape                                                                        
band                B4           B5           B6           B7           B8   
2017-04-01  422.891561  1094.207824  3108.089837  3732.298612  3999.326764   
2017-05-01  417.478271  1125.957264  3312.175030  4058.373258  4197.780260   
2017-06-01  551.242964  1301.592959  3402.514986  4113.497425  4226.873012   
2017-07-01  482.459567  1183.826676  3112.659109  3686.713608  3856.484979   
2017-08-01         NaN          NaN          NaN          NaN          NaN   
...                ...          ...          ...          ...          ...   
2024-06-01  560.437081  1284.787768  3283.141528  3997.971013  4233.961275   
2024-07-01  572.059202  1267.901809  2969.523933  3622.844338  3733.505384   
2024-08-01  680.000191  1275.776706  2751.885949  3273.287014  3445.482901   
2024-09-01  615.935952  1222.373488  2649.960805  3127.621285  3371.289760   
2024-10-01  492.556606  1098.341848  2605.414783  3036.303431  3245.904375   

greenbelt   ...         York                                                   \
shape       ...    E07000167                                                    
band        ...          B12          B2          B3          B4           B5   
2017-04-01  ...          NaN         NaN         NaN         NaN          NaN   
2017-05-01  ...  1182.111125  343.276927  627.370944  507.285913  1045.571895   
2017-06-01  ...          NaN         NaN         NaN         NaN          NaN   
2017-07-01  ...          NaN         NaN         NaN         NaN          NaN   
2017-08-01  ...          NaN         NaN         NaN         NaN          NaN   
...         ...          ...         ...         ...         ...          ...   
2024-06-01  ...  1428.124555  520.649828  833.662868  730.455500  1356.551093   
2024-07-01  ...  1348.621849  449.142528  756.872421  695.951066  1293.446824   
2024-08-01  ...  1337.656668  628.183908  850.845034  855.063460  1321.169856   
2024-09-01  ...  1478.087273  473.549769  692.731994  710.134110  1145.907284   
2024-10-01  ...  1367.883743  578.310517  805.934190  751.391124  1207.402365   

greenbelt                                                                    
shape                                                                        
band                 B6           B7           B8          B8A           B9  
2017-04-01          NaN          NaN          NaN          NaN          NaN  
2017-05-01  2744.188250  3335.098446  3509.033587  3544.669321  3575.736009  
2017-06-01          NaN          NaN          NaN          NaN          NaN  
2017-07-01          NaN          NaN          NaN          NaN          NaN  
2017-08-01          NaN          NaN          NaN          NaN          NaN  
...          

In [3]:
lookup_path = DIR_DATA / "id_lookup/id_lookup.json"
with open(lookup_path, "r") as in_file:
    D_lookup = json.load(in_file)

In [4]:
# create weighting dataframe

# instantiate empty df
df_weights = pd.DataFrame(np.nan, index=df_data.index, columns=df_data.columns )

# apply weighting
for code, dct in D_lookup.items():
    # print(code)
    area = dct["Area_ha"]

    greenbelt_name = D_lookup[code] ["GB_Name"]
    greenbelt_area = float(np.sum([val["Area_ha"] for val in D_lookup.values() if val["GB_Name"] == greenbelt_name]))

    df_weights.loc[:, (slice(None), code, slice(None))] = area/greenbelt_area


df_weights.columns = pd.MultiIndex.from_tuples([(i[0].replace("-", " "), i[1], i[2]) for i in df_weights.columns])
df_weights.columns.names = ["greenbelt", "shape", "band"]
df_weights.head()

greenbelt  Bath and Bristol                                                    \
shape             E06000022                                                     
band                     B1       B11       B12        B2        B3        B4   
2017-04-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-05-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-06-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-07-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-08-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   

greenbelt                                           ...      York            \
shape                                               ... E07000167             
band              B5        B6        B7        B8  ...       B12        B2   
2017-04-01  0.345098  0.345098  0.345098  0.345098  ...  0.031073  0.031073   
2017-05-01  0.345098  0.345098  0.345098  0.345098  ...  0.031073  0.031073   
2017-06-01  0.345098  0.345098  0.345098  0.345098  ...  0.031073  0.031073   
2017-07-01  0.345098  0.345098  0.345098  0.345098  ...  0.031073  0.031073   
2017-08-01  0.345098  0.345098  0.345098  0.345098  ...  0.031073  0.031073   

greenbelt                                                               \
shape                                                                    
band              B3        B4        B5        B6        B7        B8   
2017-04-01  0.031073  0.031073  0.031073  0.031073  0.031073  0.031073   
2017-05-01  0.031073  0.031073  0.031073  0.031073  0.031073  0.031073   
2017-06-01  0.031073  0.031073  0.031073  0.031073  0.031073  0.031073   
2017-07-01  0.031073  0.031073  0.031073  0.031073  0.031073  0.031073   
2017-08-01  0.031073  0.031073  0.031073  0.031073  0.031073  0.031073   

greenbelt                       
shape                           
band             B8A        B9  
2017-04-01  0.031073  0.031073  
2017-05-01  0.031073  0.031073  
2017-06-01  0.031073  0.031073  
2017-07-01  0.031073  0.031073  
2017-08-01  0.031073  0.031073  

[5 rows x 2160 columns]

In [5]:
# check all greenbelts for each band sum to 1.0
assert np.isclose(df_weights.groupby(level=["greenbelt", "band"], axis=1).sum(), 1.0, atol=1.e-6).all().all()
df_weights.groupby(level=["greenbelt", "band"], axis=1).sum()

/tmp/ipykernel_15612/4025412648.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  assert np.isclose(df_weights.groupby(level=["greenbelt", "band"], axis=1).sum(), 1.0, atol=1.e-6).all().all()
/tmp/ipykernel_15612/4025412648.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_weights.groupby(level=["greenbelt", "band"], axis=1).sum()


greenbelt  Bath and Bristol                                               ...  \
band                     B1  B11  B12   B2   B3   B4   B5   B6   B7   B8  ...   
2017-04-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2017-05-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2017-06-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2017-07-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2017-08-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
...                     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2024-06-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2024-07-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2024-08-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2024-09-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2024-10-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   

greenbelt  York                                               
band        B12   B2   B3   B4   B5   B6   B7   B8  B8A   B9  
2017-04-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2017-05-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2017-06-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2017-07-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2017-08-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
2024-06-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2024-07-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2024-08-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2024-09-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2024-10-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  

[91 rows x 192 columns]

In [6]:
# save DataFrame
output_path = DIR_DATA / "processed_data/sentinel_two_weights.hdf"
df_weights.to_hdf(output_path, key="df", mode="w", format="table")